In [1]:
from tqdm import tqdm
import numpy as np

from cluster_gnn.data import convert

Reference dataset taken from https://zenodo.org/record/3981290#.YIgTCi9Q3xW.

Description:
* 13 TeV collision data simulated with pythia 8.183.
* wboson.txt contains events generated from a W' boson with a mass of 600 GeV, which decays 100% of the time to a W boson and a Z boson. The W boson is forced to decay haronically and the Z boson decays into neutrinos.
* qstar.txt contains events generated from a excited quark q* with a mass of 600 GeV, which decays 100% of the time to a quark and a Z boson. The Z boson is forced to decay into neutrinos.
* events in the text format
* each line in the text represent one event, which contains variable number of detector-stable particles.
* each particle contains 7 features in order: [px, py, pz, E, pdgID, is-from-W, is-in-leading-jet]. The first four features are the four momentum of the particle, and pdgID is the pag number of the particle. is-from-W is 1 if the particle coming from W boson and 0 otherwise. is-in-leading-jet is 1 if the particle is inside the leading jet reconstructed from the anti-kT jet algorithm (R=1.0)

In [2]:
with convert.DataWriter('../data/processed/test3.hdf5') as f_out:
    with f_out.new_process('wboson') as process:
        process.decay(in_pcls=(2212, 2212), out_pcls=(23, 24))
        process.signal_id(signal_pcl=24)
        process.com_energy(energy=13.0, unit='TeV')
        with open('../data/external/wboson.txt', 'r') as f_in:
            for evt_num, line in enumerate(tqdm(f_in)):
                data = np.fromstring(line, sep=' ') # flattened data for evt
                num_cols = 7 # specified in description
                num_pcls = len(data) // num_cols
                data = data.reshape((num_pcls, num_cols))
                with process.new_event() as event:
                    event.pmu(data[:, :4])
                    event.pdg(data[:, 4])
                    event.is_signal(data[:, 5].astype(np.bool_))

100000it [03:33, 468.03it/s]
